# Motivating Recursion: When Loops Get Tedious

In [1]:
def list_sum(L):
    """Return the sum of the elements in a list."""

    total = 0
    for x in L:
        total += x
    return total

In [2]:
assert list_sum([]) == 0
assert list_sum([1]) == 1
assert list_sum([1, 2, 3]) == 6
assert list_sum([1, -1, 2, -2, 100, -100, 6]) == 6

In [3]:
def sum_all(L):
    """Return the sum of all elements in a list with arbitrary nesting."""

    total = 0
    while len(L) > 0:
        if isinstance(L[0], list):
            # Oh, the first value is a list? 
            # Rather then adding it to the total, let's smoosh it together with the input.
            L = L[0] + L[1:]
        else:
            # The first value must be something we add directly into our total.
            total += L[0]
            del L[0] # Remove first list element.
    return total

In [4]:
assert sum_all([1, 2, 3]) == 6
assert sum_all([1, [2, 3], 4]) == 10
assert sum_all([1, 2, [3, [4, [[[5]]]], 6, [7, 8]], 9]) == 45

In [5]:
def weighted_sum_all(L):
    """Return the sum of all elements in a list with arbitrary nesting, with a catch:
    Every addition to the sum should be *multiplied* by the depth where it appears.
    The first level of list has depth 1, then the next level has depth 2, etc."""

    total = 0
    multiplier = 1
    while len(L) > 0:
        if isinstance(L[0], list):
            # Oh, the first value is a list?  We got more nested, so bump multiplier.
            multiplier += 1
            # Rather then adding the nested list to the total, let's smoosh it together with the input.
            # We also leave a reminder of when to decrease the multiplier again.
            L = L[0] + ['decrease'] + L[1:]
        elif L[0] == 'decrease':
            multiplier -= 1
            del L[0]
        else:
            # The first value must be something we add directly into our total.
            total += multiplier * L[0]
            del L[0]
    return total

In [6]:
assert weighted_sum_all([1]) == 1
assert weighted_sum_all([[1]]) == 2
assert weighted_sum_all([[[1]]]) == 3
assert weighted_sum_all([[1], 3]) == 5

# Making Change

In [7]:
def make_change(amount, coins):
    """Let coins be a list of positive integer values for coins that are available for making change.
    Try to find a way to use just those coins to add up to a total of amount.
    If found, return a list of coins.  Otherwise, return None.
    """
    if amount == 0:
        # No coins, but also no more change left to make!  Easy solution.
        return []
    elif len(coins) == 0:
        # We have some change left to make, but no coins are available.
        return None
    else:
        # At least one coin remains.  Consider the cases we use it or don't use it.
        do_not_use_it = make_change(amount, coins[1:])
        if do_not_use_it != None:
            # Any solution that skips this coin is also legal if this coin was allowed, too.
            return do_not_use_it
        else:
            use_it = make_change(amount - coins[0], coins[1:])
            if use_it != None:
                # Any solution that skips this coin can be converted back to a global solution by adding the coin.
                # Note that we decreased amount in the recursive call, to ensure this trick works.
                return [coins[0]] + use_it
            else:
                return None

In [8]:
assert make_change(0, [1, 5, 10]) == []
assert make_change(15, [1, 5, 10]) == [5, 10]
assert make_change(11, [1, 5, 10]) == [1, 10]
assert make_change(13, [1, 5, 10]) == None

# Recursion and Trees

An abstract syntax tree (AST) is a convenient way of representing an arbitrary expression involving numbers and arithmetic operators.  Here's a description of an AST in Backus-Naur Form (BNF), a famous way of describing computer languages:

`ast :== integer | string | (operator, ast, ast)
operator :== "+" | "*" | …`

Here are the rules for determining the value of an AST, given an *environment* (a dictionary mapping variable names to values):

* If AST is an integer, return its value.
* If AST is a string, treat as a variable name, returning the variable's value.
* Otherwise, return the result of performing the operation on the values of the operands.

In [9]:
def eval_operator(operator, operand1, operand2):
    """Return the result of applying the given operator to the two numeric operands."""

    if operator == '+':
        return operand1 + operand2
    elif operator == '-':
        return operand1 - operand2
    elif operator == '*':
        return operand1 * operand2
    elif operator == '/':
        return operand1 / operand2
    else:
        raise ValueError

def eval_ast(ast, env={}):
    """Return the value of the AST in the given environment (dictionary mapping variables to values)."""

    if isinstance(ast, tuple):
        # If it's a tuple, it represents an operator.  Unpack the pieces and recurse.
        operator, operand1, operand2 = ast
        return eval_operator(operator, eval_ast(operand1, env), eval_ast(operand2, env))
    elif isinstance(ast, str):
        # It's a variable, represented as the variable name (string)?  Just look up in the environment.
        return env[ast]
    else:
        # It's a constant?  Then the AST is its own value.
        return ast

In [10]:
assert eval_ast(3) == 3
assert eval_ast('pi', {'pi': 3.141592653}) == 3.141592653
assert eval_ast(('*', 2, 21)) == 42
    
# evaluate 3 + 47*5/pi - 13
ast = ('-',
       ('+',
          ('/',
           ('*', 47, 5),
           'pi'),
          3),
       13)
assert eval_ast(ast, {'pi': 3.141592653}) == 64.80282326723406

# Merge Sort

In [11]:
def merge(left, right):
    """Assumes left and right are sorted lists.  Returns a single new                    
    list built, in order, from the elements of left and right."""

    result = []
    i, j = 0, 0
    # Loop while there are elements in both lists.                                     
    while i < len(left) and j < len(right):
        # Copy smallest element to result.
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Copy over any remaining elements.
    # Only one of the lists has any elements remaining!
    result.extend(left[i:]) # Recall that extend adds all elements of the argument list.
    result.extend(right[j:])
    return result

In [12]:
assert(merge([1, 2], [3, 4])) == [1, 2, 3, 4]
assert(merge([1, 3], [2, 4])) == [1, 2, 3, 4]

In [13]:
def sort(L):
    """Returns a new sorted list containing the same elements as L."""
    if len(L) <= 1:
        return L[:]
    middle = len(L) // 2
    left = sort(L[:middle])
    right = sort(L[middle:])
    return merge(left, right)

In [14]:
assert sort([23, 3, 45, 7, 6, 11, 14, 12]) == [3, 6, 7, 11, 12, 14, 23, 45]